# Tutorial 06, case 7a: Stokes problem with Neumann control

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Omega_{obs}} |\text{curl} v|^2 dx + \frac{\alpha}{2} \int_{\Gamma_C} |\nabla_{\mathbf{t}} u|^2 ds$$
s.t.
$$\begin{cases}
- \nu \Delta v + \nabla p = f                 & \text{in } \Omega\\
                       \text{div} v = 0       & \text{in } \Omega\\
                                  v = g       & \text{on } \Gamma_{in}\\
                                  v = 0       & \text{on } \Gamma_{w}\\
                 v \cdot \mathbf{n} = u       & \text{on } \Gamma_{C}\\
                 v \cdot \mathbf{t} = 0       & \text{on } \Gamma_{C}\\
                 v \cdot \mathbf{n} = 0       & \text{on } \Gamma_{s}\\
  \nu \partial_n v \cdot \mathbf{t} = 0       & \text{on } \Gamma_{s}\\
             p n - \nu \partial_n v = 0       & \text{on } \Gamma_{N}
\end{cases}$$

where
$$\begin{align*}
& \Omega                      & \text{unit square}\\
& \Gamma_{in}                 & \text{has boundary id 1}\\
& \Gamma_{s}                  & \text{has boundary id 2}\\
& \Gamma_{N}                  & \text{has boundary id 3}\\
& \Gamma_{C}                  & \text{has boundary id 4}\\
& \Gamma_{w}                  & \text{has boundary id 5}\\
& u \in [L^2(\Gamma_C)]^2     & \text{control variable}\\
& v \in [H^1(\Omega)]^2       & \text{state velocity variable}\\
& p \in L^2(\Omega)           & \text{state pressure variable}\\
& \alpha > 0                  & \text{penalization parameter}\\
& v_d                         & \text{desired state}\\
& f                           & \text{forcing term}\\
& g                           & \text{inlet profile}\\
\end{align*}$$
using an adjoint formulation solved by a one shot approach.

The test case is from section 5 of
```
F. Negri, A. Manzoni and G. Rozza. Reduced basis approximation of parametrized optimal flow control problems for the Stokes equations. Computer and Mathematics with Applications, 69(4):319-336, 2015.
```

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from ufl import as_vector, curl, div, FacetNormal, grad, inner, Measure, replace, TestFunction, TrialFunction
from dolfinx.fem import (Constant, DirichletBC, Function, FunctionSpace, locate_dofs_topological,
                         VectorFunctionSpace)
from dolfinx.io import XDMFFile
from dolfinx.plot import create_vtk_topology
from multiphenicsx.fem import (assemble_matrix_block, assemble_scalar, assemble_vector_block,
                               BlockVecSubVectorWrapper, create_vector_block, DofMapRestriction)
import pyvista

### Mesh

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/vorticity_reduction.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    boundaries = infile.read_meshtags(mesh, name="boundaries")

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

In [ ]:
# Normal and tangent
n = FacetNormal(mesh)
t = as_vector([n[1], -n[0]])

In [ ]:
def dolfinx_to_pyvista_mesh(mesh):
    num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
    cell_entities = np.arange(num_cells, dtype=np.int32)
    pyvista_cells, cell_types = create_vtk_topology(mesh, mesh.topology.dim, cell_entities)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, mesh.geometry.x)
    return grid

In [ ]:
def pyvista_mesh_plot(mesh):
    grid = dolfinx_to_pyvista_mesh(mesh)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_mesh_plot(mesh)

### Function spaces

In [ ]:
Y_velocity = VectorFunctionSpace(mesh, ("Lagrange", 2))
Y_pressure = FunctionSpace(mesh, ("Lagrange", 1))
U = FunctionSpace(mesh, ("Lagrange", 2))
L = U.clone()
Q_velocity = Y_velocity.clone()
Q_pressure = Y_pressure.clone()

### Restrictions

In [ ]:
dofs_Y_velocity = np.arange(0, Y_velocity.dofmap.index_map.size_local + Y_velocity.dofmap.index_map.num_ghosts)
dofs_Y_pressure = np.arange(0, Y_pressure.dofmap.index_map.size_local + Y_pressure.dofmap.index_map.num_ghosts)
dofs_U = locate_dofs_topological(U, boundaries.dim, boundaries.indices[boundaries.values == 4])
dofs_L = dofs_U
dofs_Q_velocity = dofs_Y_velocity
dofs_Q_pressure = dofs_Y_pressure
restriction_Y_velocity = DofMapRestriction(Y_velocity.dofmap, dofs_Y_velocity)
restriction_Y_pressure = DofMapRestriction(Y_pressure.dofmap, dofs_Y_pressure)
restriction_U = DofMapRestriction(U.dofmap, dofs_U)
restriction_L = DofMapRestriction(L.dofmap, dofs_L)
restriction_Q_velocity = DofMapRestriction(Q_velocity.dofmap, dofs_Q_velocity)
restriction_Q_pressure = DofMapRestriction(Q_pressure.dofmap, dofs_Q_pressure)
restriction = [restriction_Y_velocity, restriction_Y_pressure, restriction_U,
               restriction_L, restriction_Q_velocity, restriction_Q_pressure]

### Trial and test functions

In [ ]:
(v, p) = (TrialFunction(Y_velocity), TrialFunction(Y_pressure))
(w, q) = (TestFunction(Y_velocity), TestFunction(Y_pressure))
(u, l) = (TrialFunction(U), TrialFunction(L))
(r, m) = (TestFunction(U), TestFunction(L))
(z, b) = (TrialFunction(Q_velocity), TrialFunction(Q_pressure))
(s, d) = (TestFunction(Q_velocity), TestFunction(Q_pressure))

 ### Problem data

In [ ]:
def non_zero_eval(x):
    values = np.zeros((2, x.shape[1]))
    values[0, :] = 2.5
    return values


nu = 1.
alpha = 1.e-2
ff = Constant(mesh, (0., 0.))
bc0 = Function(Y_velocity)
bc0_component = Function(Y_velocity.sub(0).collapse())
bc1 = Function(Y_velocity)
bc1.interpolate(non_zero_eval)

### Optimality conditions

In [ ]:
def vorticity(v, w):
    return inner(curl(v), curl(w))


def penalty(u, r):
    return alpha * inner(grad(u), t) * inner(grad(r), t)


a = [[vorticity(v, w) * dx(4), None, None, l * inner(w, n) * ds(4), nu * inner(grad(z), grad(w)) * dx,
      - b * div(w) * dx],
     [None, None, None, None, - q * div(z) * dx, None],
     [None, None, penalty(u, r) * ds(4), -l * r * ds(4), None, None],
     [m * inner(v, n) * ds(4), None, - m * u * ds(4), None, None, None],
     [nu * inner(grad(v), grad(s)) * dx, - p * div(s) * dx, None, None, None, None],
     [- d * div(v) * dx, None, None, None, None, None]]
f = [None,
     None,
     None,
     None,
     inner(ff, s) * dx,
     None]
a[0][0] += Constant(mesh, 0.) * inner(v, w) * dx
a[4][4] = Constant(mesh, 0.) * inner(z, s) * dx
f[0] = inner(Constant(mesh, (0., 0.)), w) * dx
f[1] = Constant(mesh, 0.) * q * dx
f[2] = Constant(mesh, 0.) * r * dx
f[3] = Constant(mesh, 0.) * m * dx
f[5] = Constant(mesh, 0.) * d * dx


def bdofs(space_from, space_to, idx):
    return locate_dofs_topological((space_from, space_to), mesh.topology.dim - 1,
                                   boundaries.indices[boundaries.values == idx])


bc = [DirichletBC(bc1, bdofs(Y_velocity, bc1.function_space, 1), Y_velocity),
      DirichletBC(bc0_component, bdofs(Y_velocity.sub(1), bc0_component.function_space, 2),
                  Y_velocity.sub(1)),
      DirichletBC(bc0_component, bdofs(Y_velocity.sub(0), bc0_component.function_space, 4),
                  Y_velocity.sub(0)),
      DirichletBC(bc0, bdofs(Y_velocity, bc0.function_space, 5), Y_velocity),
      DirichletBC(bc0, bdofs(Q_velocity, bc0.function_space, 1), Q_velocity),
      DirichletBC(bc0_component, bdofs(Q_velocity.sub(1), bc0_component.function_space, 2),
                  Q_velocity.sub(1)),
      DirichletBC(bc0, bdofs(Q_velocity, bc0.function_space, 4), Q_velocity),
      DirichletBC(bc0, bdofs(Q_velocity, bc0.function_space, 5), Q_velocity)]

### Solution

In [ ]:
(v, p) = (Function(Y_velocity), Function(Y_pressure))
(u, l) = (Function(U), Function(L))
(z, b) = (Function(Q_velocity), Function(Q_pressure))

### Cost functional

In [ ]:
J = 0.5 * vorticity(v, v) * dx(4) + 0.5 * penalty(u, u) * ds(4)

### Uncontrolled functional value

In [ ]:
# Extract state forms from the optimality conditions
a_state = [[replace(a[i][j], {s: w, d: q}) if a[i][j] is not None else None
            for j in (0, 1)] for i in (4, 5)]
f_state = [replace(f[i], {s: w, d: q}) for i in (4, 5)]
bc_state = [DirichletBC(bc1, bdofs(Y_velocity, bc1.function_space, 1), Y_velocity),
            DirichletBC(bc0_component, bdofs(Y_velocity.sub(1), bc0_component.function_space, 2),
                        Y_velocity.sub(1)),
            DirichletBC(bc0, bdofs(Y_velocity, bc0.function_space, 4), Y_velocity),
            DirichletBC(bc0, bdofs(Y_velocity, bc0.function_space, 5), Y_velocity)]

In [ ]:
# Assemble the block linear system for the state
A_state = assemble_matrix_block(a_state, bcs=bc_state,
                                restriction=([restriction[i] for i in (4, 5)],
                                             [restriction[j] for j in (0, 1)]))
A_state.assemble()
F_state = assemble_vector_block(f_state, a_state, bcs=bc_state,
                                restriction=[restriction[i] for i in (4, 5)])

In [ ]:
# Solve
vp = create_vector_block([f[j] for j in (0, 1)], restriction=[restriction[j] for j in (0, 1)])
ksp = PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A_state)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F_state, vp)
vp.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with BlockVecSubVectorWrapper(vp, [c.function_space.dofmap for c in (v, p)]) as vp_wrapper:
    for vp_wrapper_local, component in zip(vp_wrapper, (v, p)):
        with component.vector.localForm() as component_local:
            component_local[:] = vp_wrapper_local

In [ ]:
J_uncontrolled = mesh.comm.allreduce(assemble_scalar(J), op=MPI.SUM)
print("Uncontrolled J =", J_uncontrolled)
assert np.isclose(J_uncontrolled, 2.9238625)

In [ ]:
def pyvista_scalar_field_plot(mesh, scalar_field, name):
    grid = dolfinx_to_pyvista_mesh(mesh)
    grid.point_arrays[name] = scalar_field.compute_point_values()
    grid.set_active_scalars(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
def pyvista_vector_field_plot(mesh, vector_field, name, factor):
    grid = dolfinx_to_pyvista_mesh(mesh)
    values = np.zeros((mesh.geometry.x.shape[0], 3))
    values[:, :2] = vector_field.compute_point_values()
    grid.point_arrays[name] = values
    grid.set_active_vectors(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    glyphs = grid.glyph(orient=name, factor=factor)
    plotter.add_mesh(glyphs)
    plotter.show()

In [ ]:
pyvista_vector_field_plot(mesh, v, "uncontrolled state velocity", factor=3e-2)

In [ ]:
pyvista_scalar_field_plot(mesh, p, "uncontrolled state pressure")

### Optimal control

In [ ]:
# Assemble the block linear system for the optimality conditions
A = assemble_matrix_block(a, bcs=bc, restriction=(restriction, restriction))
A.assemble()
F = assemble_vector_block(f, a, bcs=bc, restriction=restriction)

In [ ]:
# Solve
vpulzb = create_vector_block(f, restriction=restriction)
ksp = PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, vpulzb)
vpulzb.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with BlockVecSubVectorWrapper(vpulzb, [c.function_space.dofmap for c in (v, p, u, l, z, b)],
                              restriction) as vpulzb_wrapper:
    for vpulzb_wrapper_local, component in zip(vpulzb_wrapper, (v, p, u, l, z, b)):
        with component.vector.localForm() as component_local:
            component_local[:] = vpulzb_wrapper_local

In [ ]:
J_controlled = mesh.comm.allreduce(assemble_scalar(J), op=MPI.SUM)
print("Optimal J =", J_controlled)
assert np.isclose(J_controlled, 1.71054295)

In [ ]:
pyvista_vector_field_plot(mesh, v, "state velocity", factor=3e-2)

In [ ]:
pyvista_scalar_field_plot(mesh, p, "state pressure")

In [ ]:
pyvista_vector_field_plot(mesh, u, "control", factor=1e-1)

In [ ]:
pyvista_vector_field_plot(mesh, l, "lambda", factor=1e-3)

In [ ]:
pyvista_vector_field_plot(mesh, z, "adjoint velocity", factor=1)

In [ ]:
pyvista_scalar_field_plot(mesh, b, "adjoint pressure")